## Spotify App Reviews Sentiment Analysis

In [10]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from wordcloud import WordCloud

In [11]:
stf = pd.read_csv('reviews.csv')

### Data PreProcessing

In [4]:
!pip install nltk gensim

Defaulting to user installation because normal site-packages is not writeable
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.5/1.5 MB 160.7 kB/s eta 0:00:00m eta 0:00:010:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 298.0/298.0 KB 75.2 kB/s eta 0:00:00m eta 0:00:010:01:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 770.5/770.5 KB 160.4 kB/s eta 0:00:00m eta 0:00:010:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.5/78.5 KB 84.5 kB/s eta 0:00:00 kB/s eta 0:00:01:01


In [12]:
import gensim
from gensim.parsing.preprocessing import strip_non_alphanum, strip_multiple_whitespaces, preprocess_string, split_alphanum, strip_short, strip_numeric
import re 
import nltk
from nltk.corpus import stopwords
nltk.download('stopwords')
stop_words = stopwords.words('english')

[nltk_data] Downloading package stopwords to /home/canada/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [13]:
def lower_case(docs):
  return [doc.lower() for doc in docs]

# Loai bo dau cau & ki tu dac biet
def remove_punc(docs):
  return [strip_non_alphanum(doc).strip() for doc in docs]

# Tach cac so va chu lien nhau
def separate_num(docs):
  return [split_alphanum(doc) for doc in docs]

def remove_one_letter_word(docs):
  return [strip_short(doc) for doc in docs]

def remove_number(docs):
  return [strip_numeric(doc) for doc in docs]

# Thay the nhieu spaces bang mot space
def replace_multiple_whitespaces(docs):
  return [strip_multiple_whitespaces(doc) for doc in docs]

def remove_stopwords(docs):
    return [" ".join([word for word in doc.split() if word not in stop_words]) for doc in docs]

doc = lower_case(stf['Review'])
doc = remove_punc(doc)
doc = separate_num(doc)
doc = remove_one_letter_word(doc)
doc = remove_number(doc)
doc = replace_multiple_whitespaces(doc)
doc = remove_stopwords(doc)

stf['Review'] = doc

In [16]:
stf.head()

,Time_submitted,Review,Rating,Total_thumbsup,Reply
0,2022-07-09 15:00:00,great music service audio high quality app eas...,5,2,NaN
1,2022-07-09 14:21:22,please ignore previous negative rating app sup...,5,1,NaN
2,2022-07-09 13:27:32,pop get best spotify experience android annoyi...,4,0,NaN
3,2022-07-09 13:26:45,really buggy terrible use recently,1,1,NaN
4,2022-07-09 13:20:49,dear spotify get songs put playlist shuffle play,1,1,NaN


In [1]:
stf1 = stf.copy()
rate = [1,2,3,4,5]
sentiment = ['negative', 'negative', 'positive', 'positive', 'positive']
for i in range(5):
    stf1["Rating"].replace(rate[i], sentiment[i],inplace=True)

NameError: name 'stf' is not defined

In [17]:
stf1.head()

,Time_submitted,Review,Rating,Total_thumbsup,Reply
0,2022-07-09 15:00:00,great music service audio high quality app eas...,positive,2,NaN
1,2022-07-09 14:21:22,please ignore previous negative rating app sup...,positive,1,NaN
2,2022-07-09 13:27:32,pop get best spotify experience android annoyi...,positive,0,NaN
3,2022-07-09 13:26:45,really buggy terrible use recently,negative,1,NaN
4,2022-07-09 13:20:49,dear spotify get songs put playlist shuffle play,negative,1,NaN


### RNN

In [32]:
!pip install --upgrade pip

Defaulting to user installation because normal site-packages is not writeable
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 10.1 MB/s eta 0:00:00 MB/s eta 0:00:01:01


In [38]:
!pip3 install --upgrade protobuf==3.20.0

Defaulting to user installation because normal site-packages is not writeable
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 6.8 MB/s eta 0:00:00m eta 0:00:010:00:01
  Attempting uninstall: protobuf
    Found existing installation: protobuf 3.20.3
    Uninstalling protobuf-3.20.3:
      Successfully uninstalled protobuf-3.20.3
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tensorflow 2.11.0 requires protobuf<3.20,>=3.9.2, but you have protobuf 3.20.0 which is incompatible.
ortools 9.5.2237 requires protobuf>=4.21.5, but you have protobuf 3.20.0 which is incompatible.
grpcio-status 1.50.0 requires protobuf>=4.21.6, but you have protobuf 3.20.0 which is incompatible.
google-cloud-bigquery 3.3.6 requires protobuf!=3.20.0,!=3.20.1,!=4.21.0,!=4.21.1,!=4.21.2,!=4.21.3,!=4.21.4,!=4.21.5,<5.0.0dev,>=3.19.5, but you have protobuf 3.20.0 which is incompatible.

In [22]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

In [26]:


# Prepare the dataset
X_train, X_test, y_train, y_test = train_test_split(stf1['Review'], stf1['Rating'], test_size=0.2, random_state=42)
print(X_train[0])

# Tokenize the data
tokenizer = Tokenizer()
tokenizer.fit_on_texts(X_train)
print(X_train[0])
X_train = tokenizer.texts_to_sequences(X_train)
print(X_train[0])

# Pad the sequences to make all inputs the same length
max_length = max([len(s.split()) for s in X_train])
X_train = pad_sequences(X_train, maxlen=max_length, padding='post')

# Define the model
model = keras.Sequential([
    keras.layers.Embedding(input_dim=len(tokenizer.word_index)+1, output_dim=32, input_length=max_length),
    keras.layers.SimpleRNN(units=32),
    keras.layers.Dense(units=3, activation='softmax')
])

# Compile the model
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# Train the model
model.fit(X_train, y_train, epochs=10)

# Make predictions on the test data
X_test = [
    "This is a positive review",
    "This is a negative review",
    "This is a neutral review"
]
X_test = tokenizer.texts_to_sequences(X_test)
X_test = pad_sequences(X_test, maxlen=max_length, padding='post')
y_pred = model.predict_classes(X_test)

great music service audio high quality app easy use also quick friendly support
great music service audio high quality app easy use also quick friendly support
[816, 3, 13, 71, 162, 1062, 1418, 2717, 516, 317, 285, 61]


AttributeError: 'list' object has no attribute 'split'

### Random Forests

In [18]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics import accuracy_score
from sklearn.ensemble import RandomForestClassifier
from sklearn.naive_bayes import MultinomialNB

In [19]:
X_train, X_test, y_train, y_test = train_test_split(stf1['Review'], stf1['Rating'], test_size=0.2, random_state=42)

In [20]:
tfidf = TfidfVectorizer()
tfidf.fit(X_train)
X_train_tfidf = tfidf.transform(X_train)
X_test_tfidf = tfidf.transform(X_test)

In [21]:
rf2 = RandomForestClassifier()
rf2.fit(X_train_tfidf, y_train)
rf_tfidf = rf2.score(X_test_tfidf, y_test)
print(f'Random Forest Classifier on TF-IDF Vectors: {rf_tfidf}')

Random Forest Classifier on TF-IDF Vectors: 0.8152447438915497


In [24]:
mnb2 = MultinomialNB()
mnb2.fit(X_train_tfidf, y_train)
mnb_tfidf = mnb2.score(X_test_tfidf, y_test)
print(f'Multinomial Naive Bayes Classifier on TF-IDF Vectors: {mnb_tfidf}')

Multinomial Naive Bayes Classifier on TF-IDF Vectors: 0.8098871661660849


In [25]:
model = {'Model':['RandomForestClassifier-TFIDFVectors', 
                  'MultinomialNBClassifier-TFIDFVectors'],
         'Score':[rf_tfidf,mnb_tfidf]
         }
model_df = pd.DataFrame(model)
model_df

,Model,Score
0,RandomForestClassifier-TFIDFVectors,0.814027
1,MultinomialNBClassifier-TFIDFVectors,0.809887
